In [1]:
import torch
print(torch.cuda.is_available() , torch.__version__)

False 2.2.0+cpu


In [1]:
# %% test a specific model
from src.interface import ModelTrainer
ModelTrainer.main(stage = 0 , resume = 0 , checkname= 1)

24-05-02 08:35:53|MOD:display     |: Model Specifics:
24-05-02 08:35:53|MOD:display     |: Start Process [Data] at Thu May  2 08:35:53 2024!


--Process Queue : Data + Fit + Test
--Model_name is set to gru_day_ShortTest!
Callback : DynamicDataLink() , assign / unlink dynamic data in tra networks
Callback : ResetOptimizer(num_reset=2,trigger=40,recover_level=1.0,speedup2x=True) , reset optimizer on some epoch (can speedup scheduler)
Callback : CallbackTimer() , record time cost of callback hooks
Callback : EarlyStoppage(patience=20) , stop fitting when validation score cease to improve
Callback : ValidationConverge(patience=5,eps=1e-05) , stop fitting when valid_score converge
Callback : EarlyExitRetrain(earliest=20,max_attempt=4,lr_multiplier=[1, 0.1, 10, 0.01, 100, 1]) , retrain with new lr if fitting stopped too early
Callback : NanLossRetrain(max_attempt=4) , retrain if fitting encounters nan loss
Callback : BatchDisplay(verbosity=10) , display batch progress bar
Callback : StatusDisplay(verbosity=10) , display epoch / event information
{'random_seed': None,
 'model_name': 'gru_day_ShortTest',
 'model_module': 'gru',
 'mod

24-05-02 08:35:55|MOD:display     |: Finish Process [Data], Cost 1.7 Secs
24-05-02 08:35:55|MOD:display     |: Start Process [Fit] at Thu May  2 08:35:55 2024!


Pre-Norming method of [day] : {'divlast': True, 'histnorm': True}


  0%|          | 0/10 [00:00<?, ?it/s]

score function of [spearman] calculated and success!
loss function of [pearson] calculated and success!


Valid Ep#  0 score : -0.05205: 100%|██████████| 10/10 [00:02<00:00,  4.44it/s]
FirstBite Ep#  0 : loss  1.01875, train-0.02000, valid-0.05205, best-0.0521, lr1.3e-03
Valid Ep#  1 score : 0.13114: 100%|██████████| 10/10 [00:02<00:00,  4.54it/s]
FirstBite Ep#  1 : loss  0.96613, train 0.03572, valid 0.13114, best 0.1311, lr2.5e-03
Valid Ep#  2 score : 0.09193: 100%|██████████| 10/10 [00:02<00:00,  4.61it/s]
FirstBite Ep#  2 : loss  0.92229, train 0.08836, valid 0.09193, best 0.1311, lr3.8e-03
Valid Ep#  3 score : 0.12176: 100%|██████████| 10/10 [00:02<00:00,  3.80it/s]
FirstBite Ep#  3 : loss  0.89578, train 0.11877, valid 0.12176, best 0.1311, lr5.0e-03
24-05-02 08:37:53|MOD:display     |: gru_day_ShortTest #0 @20170103|FirstBite Ep#  4 Max Epoch|Train 0.1188 Valid 0.1218 BestVal 0.1311|Cost  1.9Min, 23.3Sec/Ep
24-05-02 08:37:53|MOD:display     |: Finish Process [Fit], Cost 0.0 Hours, 2.0 Min/model, 29.5 Sec/Epoch
24-05-02 08:37:53|MOD:display     |: Start Process [Test] at Thu May  2 0

                    hook_name  num_calls  total_time   avg_time
8              on_train_batch         40   86.657342   2.166434
23              on_test_batch        105   26.707520   0.254357
17           on_fit_model_end          1   20.285721  20.285721
13        on_validation_batch         40    9.182415   0.229560
4          on_fit_model_start          1    1.676516   1.676516
20        on_test_model_start          1    0.586431   0.586431
24          on_test_batch_end        105    0.111710   0.001064
14    on_validation_batch_end         40    0.044900   0.001122
9          on_train_batch_end         40    0.040891   0.001022
21   on_test_model_type_start          3    0.032912   0.010971
16           on_fit_epoch_end          4    0.030916   0.007729
27                on_test_end          1    0.015958   0.015958
15    on_validation_epoch_end          4    0.012967   0.003242
6        on_train_epoch_start          4    0.006020   0.001505
11  on_validation_epoch_start          4

In [ ]:
from src.interface import ModelTrainer
ModelTrainer.main(stage = 0 , resume = 0 , checkname= 1)

In [1]:
# prepare data
from src.interface import DataModule
DataModule.prepare_data()

KeyboardInterrupt: 